In [2]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.openai import OpenAI
import os 
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.objects import ObjectIndex


chroma_client = chromadb.PersistentClient(path="chroma_db")
chroma_collection = chroma_client.get_or_create_collection("quickstart")


In [3]:
function_llm = OpenAI(model="gpt-4",  api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))
Settings.llm = OpenAI(model="gpt-4", api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))
Settings.embed_model = OpenAIEmbedding(api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))

In [4]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)



In [5]:
engine = index.as_query_engine(llm=Settings.llm)

response = engine.query("summarize the document for the RFP related to construction?")
print(response)

The document outlines the responsibilities of the Construction Manager and the Owner in a construction project. The Construction Manager must comply with all applicable laws, regulations, and programs. The Owner is responsible for retaining an Architect for various services related to the project, and for costs related to environmental studies, surveys, and geotechnical services. The Owner also has a designated representative to assist in decision making. The Construction Manager is required to maintain insurance as set forth in the Construction Contract. The document also provides a detailed format for proposals, which includes information about the proposer, the organization, personnel, hiring practices, qualifications and experience, sustainability experience, connection to place, and self-performed work.


In [6]:
response.metadata


{'157c5b6f-c504-4903-8cb3-b0c0a42b8169': {'page_label': '4',
  'file_name': 'kitchenai/fa102c5e-2e00-429c-922f-b7976d584b0d/consultancy-rfp.pdf',
  'file_id': 31,
  'source': 'kitchenai_cookbook',
  'file_label': 'consultancy',
  'document_title': '"Guidelines and Requirements for Contract Bidding on the Town\'s Request for Proposal (RFP)"',
  'questions_this_excerpt_can_answer': '1. What is the consequence if a bidder submits more than one proposal for the same contract according to the document from kitchenai_cookbook?\n2. Who is the designated contact person for all inquiries regarding the RFP as per the document in the kitchenai_cookbook?\n3. What is the stipulated time frame for the execution of the contract after it has been awarded according to the guidelines in the consultancy-rfp.pdf document?\n4. What is the requirement for initiating work after the execution of the contract as per the guidelines in the consultancy-rfp.pdf document?\n5. What are the documents that a bidder ne

In [7]:
file_label = "construction"

label_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    filter={"file_label": file_label}
)


In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="rfp_collection",
            description=(
                "A catch all tool for any question."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=label_index.as_query_engine(llm=Settings.llm),
        metadata=ToolMetadata(
            name=file_label,
            description=(
                f"Provides information about the current working RFP: {file_label}"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]



In [9]:
local_agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=function_llm,
    verbose=True,
    system_prompt=f"""\
You are an expert in analyzing RFPs and breaking down their requirements from previous incoming RFP requests.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
)


obj_index = ObjectIndex.from_objects(
    query_engine_tools,
    index_cls=VectorStoreIndex,
)

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an RFP expert designed to help the proposal team understand the requirements of incoming RFPs.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\
""",
    verbose=True,
)

In [10]:
response = top_agent.query(f"build a report on the requirements of the incoming RFP: construction, include a list of requirements and a comparison to the advertising and consultancy rfps")


Added user message to memory: build a report on the requirements of the incoming RFP: construction, include a list of requirements and a comparison to the advertising and consultancy rfps
=== Calling Function ===
Calling function: construction with args: {
"input": "What are the requirements of the incoming RFP: construction?"
}
Got output: The requirements for the incoming RFP: construction include demonstrating prior experience with Integrated Project / Construction Management-at-risk delivery methods, specifically completion of at least one project over $8M in construction costs. The proposer should also have prior experience with delivering projects that have achieved sustainability benchmarks such as LEED or Living Building Challenge. The proposal should include qualified personnel with experience collaborating with Architects and Engineers during Design Phases and have qualified Cost Estimators on staff. After selection, the Construction Manager should collaborate with Incourage 

In [11]:
response.response

"The requirements for the incoming RFPs are as follows:\n\n**Construction RFP:**\n1. Demonstrated prior experience with Integrated Project / Construction Management-at-risk delivery methods, specifically completion of at least one project over $8M in construction costs.\n2. Prior experience with delivering projects that have achieved sustainability benchmarks such as LEED or Living Building Challenge.\n3. Qualified personnel with experience collaborating with Architects and Engineers during Design Phases.\n4. Qualified Cost Estimators on staff.\n5. Collaboration with Incourage Community Foundation to define and elaborate upon the final scope of work.\n6. Availability to begin work immediately upon selection and preparedness for project activities to commence.\n7. Acknowledgement of ability to work within the project timeline, with understanding of potential adjustments as the project evolves.\n\n**Advertising RFP:**\n1. Demonstrated experience in journalism, interviewing, writing, and 